In [2]:
import pandas as pd 
import matplotlib.pyplot as plt


# Création du Dataset Info_rank

In [2]:
df_infos_rang_joueurs = pd.read_csv('./Data/Data_utiles/info_rank')

In [3]:
df_infos_rang_joueurs.head()


,Unnamed: 0,ranking_date,rank,player_id,points,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,134199,19900101,1,100656,2913.0,Ivan,Lendl,R,19600307.0,USA,188.0,Q182736
1,134465,19900101,2,101414,2279.0,Boris,Becker,R,19671122.0,GER,190.0,Q76334
2,134961,19900101,3,101222,2111.0,Stefan,Edberg,R,19660119.0,SWE,188.0,Q189542
3,135361,19900101,4,100763,1398.0,Brad,Gilbert,R,19610809.0,USA,185.0,Q715134
4,135627,19900101,5,100581,1354.0,John,McEnroe,L,19590216.0,USA,180.0,Q16474


In [16]:
df_infos_rang_joueurs.shape

(162982, 12)

On voit que ce dataset est composé de $12$ colonnes et $162982$ lignes, ce qui est assez conséquent. 

Le but de notre projet étant de définir des caractéristiques qui distinguent un joueur du top 10 d'un autre du top 50, nous pouvons d'ors et déjà supprimer certaines colonnes qui ne nous seront pas utiles telles que:

- IOC vu que le pays d'origine du joueur n'a aucun impact sur ses performances
- wikidata_id
- dob
- Unnamed :0 (qui est un duplicat des indexes)

In [4]:
df_infos_rang_joueurs.drop(['Unnamed: 0', 'ioc', 'wikidata_id','dob'],axis=1,inplace=True)



In [5]:
df_infos_rang_joueurs.head()

,ranking_date,rank,player_id,points,name_first,name_last,hand,height
0,19900101,1,100656,2913.0,Ivan,Lendl,R,188.0
1,19900101,2,101414,2279.0,Boris,Becker,R,190.0
2,19900101,3,101222,2111.0,Stefan,Edberg,R,188.0
3,19900101,4,100763,1398.0,Brad,Gilbert,R,185.0
4,19900101,5,100581,1354.0,John,McEnroe,L,180.0


In [6]:
df_infos_rang_joueurs.to_csv('./Data/Data_utiles/info_rank_after_treatment')

# Création du Dataset Info_match

Ce dataset va regrouper tous les matchs joués par les joueurs du top 100. On aura accès aux informations suivantes:

- Vainqueur / Perdant
- Rang du vainqueur / Rang du perdant
- surface du match
- tournois

In [8]:
liste_concat=[]

for year in range(1993,2022):
    df_matches_annee=pd.read_csv('./Data/atp_matches_{}.csv'.format(year))

    #On garde seulement les matches des joueurs du top 100

    df_matches=df_matches_annee[(df_matches_annee['winner_rank']<=100) | (df_matches_annee['loser_rank']<=100)].copy()
    
    liste_concat.append(df_matches)

info_matches=pd.concat(liste_concat)
colonnes_a_supprimer = ['winner_ioc', 'winner_entry', 'winner_seed', 'draw_size','loser_ioc', 'loser_entry', 'loser_seed']
info_matches=info_matches.drop(colonnes_a_supprimer,axis=1)

In [9]:
print(info_matches.shape)
print(info_matches.columns)

(80978, 42)
Index(['tourney_id', 'tourney_name', 'surface', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_age', 'loser_id', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_age', 'score', 'best_of', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')


In [15]:
info_matches.to_csv('./Data/Data_utiles/info_matches.csv')

### Fonction pour calculer le winrate d'un joueur sur une année

In [11]:
def winrate(year,_id,dataset):
    
    df_year = dataset[(dataset['tourney_date'] >= year * 10000) & (dataset['tourney_date'] < (year + 1) * 10000)].copy()

    nombre_matches_gagnes=0
    nombre_matches_joues=0
    for index, row in df_year.iterrows():
        
        if row['winner_id']==_id :
            nombre_matches_gagnes+=1
            nombre_matches_joues+=1
            
        elif row['loser_id']==_id:
            nombre_matches_joues+=1
    
    winrate=(nombre_matches_gagnes/nombre_matches_joues)
    
    return winrate,nombre_matches_gagnes,nombre_matches_joues


In [13]:
print(winrate(1993,101948,info_matches))

(0.8415841584158416, 85, 101)


# Création du dataset Info_joueurs

In [3]:
info_matches1=pd.read_csv('./Data/Data_utiles/info_matches.csv')
columns_needed = ['l_1stIn', 'l_svpt', 'l_1stWon', 'l_2ndWon', 'w_svpt', 'w_1stWon', 'w_2ndWon','tourney_date', 'loser_rank', 'winner_rank','loser_rank','w_bpFaced','l_bpFaced','l_bpSaved','w_bpSaved','l_SvGms','w_SvGms']

# Filtrer le DataFrame pour exclure les lignes avec des NaN dans les colonnes spécifiées
info_matches=info_matches1.dropna(subset=columns_needed)

In [ ]:
#On crée un dictionnaire qui référencie tous les joueurs

joueurs_dict = {}


for index, row in info_matches.iterrows():
    winner_id = row['winner_id']
    loser_id = row['loser_id']

    if (row['winner_rank']<=100) and (winner_id not in joueurs_dict):
        joueurs_dict[winner_id] = {'name': row['winner_name'], 'hand': row['winner_hand'], 'height': row['winner_ht']}
        for year in range(1993, 2023):  
            joueurs_dict[winner_id][str(year)] = []

    if (row['loser_rank']<=100) and (loser_id not in joueurs_dict):
        joueurs_dict[loser_id] = {'name': row['loser_name'], 'hand': row['loser_hand'], 'height': row['loser_ht']}
        for year in range(1993, 2023):  
            joueurs_dict[loser_id][str(year)] = []

In [ ]:
def calculate_percentage(value, total):
    return value / total if total != 0 else 0

def process_match(row, player_id, stats):
    stats['matchs'] += 1
    stats['win'] += 1
    stats['pourc_serv_in'] += calculate_percentage(int(row['w_1stIn']), int(row['w_svpt']))
    stats['pourc_serv_win_pnt'] += calculate_percentage(int(row['w_1stWon']) + int(row['w_2ndWon']), int(row['w_svpt']))
    stats['pourc_return_win_pnt'] += calculate_percentage(int(row['l_svpt']) - int(row['l_1stWon']) - int(row['l_2ndWon']), int(row['l_svpt']))
    stats['pourc_serv_games_win'] += calculate_percentage(int(row['w_SvGms']) - (int(row['w_bpFaced']) - int(row['w_bpSaved'])), int(row['w_SvGms']))
    stats['pourc_break_games'] += 1 - calculate_percentage(int(row['l_SvGms']) - (int(row['l_bpFaced']) - int(row['l_bpSaved'])), int(row['l_SvGms']))
    stats['mean_ranking_oppo'] += int(row['loser_rank'])

def update_stats_for_player(player_id, stats, year):
    if stats['matchs'] > 0:
        for key in ['pourc_serv_in', 'pourc_serv_win_pnt', 'pourc_return_win_pnt', 'pourc_serv_games_win', 'pourc_break_games']:
            stats[key] = calculate_percentage(stats[key], stats['matchs'])

        for key in ['pourc_break_point_saved', 'pourc_break_point_made', 'mean_ranking_oppo']:
            stats[key] = calculate_percentage(stats[key], stats['matchs'])

        joueurs_dict[player_id][str(year)] = [stats['matchs'], stats['win'], stats['pourc_return_win_pnt'],
                                              stats['pourc_return_win_pnt'], stats['pourc_break_games'],
                                              stats['pourc_break_point_made'], stats['pourc_break_point_saved'],
                                              stats['pourc_serv_games_win'], stats['pourc_serv_in']]



for year in range(1993,2021):
    for joueur in joueurs_dict.keys():
        
        stats = {
            'matchs': 0,
            'win': 0,
            'pourc_serv_win_pnt': 0,
            'pourc_return_win_pnt': 0,
            'pourc_serv_in': 0,
            'pourc_break_games': 0,
            'pourc_serv_games_win': 0,
            'pourc_break_point_saved': 0,
            'pourc_break_point_made': 0,
            'mean_ranking_oppo': 0
        }
        df_year = info_matches[(info_matches['tourney_date'] >= year * 10000) & (info_matches['tourney_date'] < (year + 1) * 10000)].copy()

        for i in range(len(df_year)):
            winner_id = df_year['winner_id'].iloc[i]
            loser_id = df_year['loser_id'].iloc[i]

            if winner_id == joueur :
                process_match(df_year.iloc[i], joueur, stats)

            elif loser_id == joueur :
                process_match(df_year.iloc[i], joueur, stats)


        update_stats_for_player(joueur, stats, year)


In [ ]:
df_info_joueurs=pd.DataFrame(joueurs_dict)
df_info_joueurs=df_info_joueurs.transpose()

df_info_joueurs.to_csv('./Data/Data_utiles/info_joueurs.csv')